In [1]:
import os,pickle
import numpy as np
import tensorflow as tf
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from process_data import preprocess_conditional_flow_data_cww
from Model.ConditionalRealNVP import ConditionalRealNVP,RealNVP
from Model.Discriminator import Discriminator
from Utils.ObjDict import ObjDict
from Utils.mkdir_p import mkdir_p

In [3]:
# __________________________________________________________________ ||
# Basic configurables
# __________________________________________________________________ ||

input_csv_path = "data/train_cww.npy"
saved_sig_model_path = "output/train_mixnlp_cww_210120_v2/saved_model_sig.h5"
saved_bkg_model_path = "output/train_mixnlp_cww_210120_v2/saved_model_bkg.h5"
saved_disc_model_path = "output/train_mixnlp_cww_210120_v2/saved_model_disc.h5"
output_dir = os.path.dirname(saved_sig_model_path)
event_size = 4000
ndim = 3
ncond = 1

In [7]:
# __________________________________________________________________ ||
# Load models
# __________________________________________________________________ ||

nf_model = ObjDict(
    sig = ConditionalRealNVP(num_coupling_layers=5,ndim=ndim,ncond=ncond),
    bkg = RealNVP(num_coupling_layers=5,ndim=ndim),
    disc = Discriminator([32,32,32,]),
)
samples = nf_model.sig.distribution.sample(event_size)
condition = 1.0 * np.ones((event_size,1))

_,_ = nf_model.sig.predict([samples,condition,])
nf_model.sig.load_weights(saved_sig_model_path)

_,_ = nf_model.bkg.predict(samples)
nf_model.bkg.load_weights(saved_bkg_model_path)

_ = nf_model.disc.predict(samples)
nf_model.disc.load_weights(saved_disc_model_path)


In [8]:
arr = np.load(input_csv_path)
sigs,bkg = preprocess_conditional_flow_data_cww(arr)

In [9]:
# __________________________________________________________________ ||
# Make plots for different conditions
# __________________________________________________________________ ||

n_dim = 5
param_grid = [sigs[idx_param] for idx_param in np.random.randint(0,len(sigs),n_dim*n_dim)]
param_grid.sort(key=lambda x: x.condition[0])
figsize = (50,50)

samples = nf_model.sig.distribution.sample(event_size)
fig_m4l,ax_m4l = plt.subplots(n_dim,n_dim,figsize=figsize)
fig_mz1,ax_mz1 = plt.subplots(n_dim,n_dim,figsize=figsize)
fig_mz2,ax_mz2 = plt.subplots(n_dim,n_dim,figsize=figsize)

for i,m in enumerate(param_grid):
    
    ix = int(i / n_dim)
    iy = i % n_dim
    
    condition_str = str(m.condition[0])
    condition = np.ones((event_size,1)) * m.condition[0]

    idx_batch = np.random.randint(0,m.x.shape[0],event_size)
    
    x_sig_true = m.x[idx_batch]
    x_sig_gen,_ = nf_model.sig.predict([samples,condition,])
    
    idx_batch = np.random.randint(0,bkg.x.shape[0],event_size)
    x_bkg_true = bkg.x[idx_batch]
    condition = np.ones((event_size,1)) * m.condition[0]
    x_bkg_gen,_ = nf_model.bkg.predict(samples)

    ax_m4l[ix,iy].hist(x_sig_true[:,0],bins=100,density=1.,histtype='step',range=[-10.,10.],label='True sig '+condition_str)
    ax_m4l[ix,iy].hist(x_sig_gen[:,0],bins=100,density=1.,histtype='step',range=[-10.,10.],label='Flow sig '+condition_str)
    ax_m4l[ix,iy].hist(x_bkg_true[:,0],bins=100,density=1.,histtype='step',range=[-10.,10.],label='True bkg'+condition_str)
    ax_m4l[ix,iy].hist(x_bkg_gen[:,0],bins=100,density=1.,histtype='step',range=[-10.,10.],label='Flow bkg '+condition_str)
    ax_m4l[ix,iy].legend(loc='best')
    ax_m4l[ix,iy].set_title(condition_str)
    
    ax_mz1[ix,iy].hist(x_sig_true[:,1],bins=100,density=1.,histtype='step',range=[-10.,10.],label='True sig '+condition_str)
    ax_mz1[ix,iy].hist(x_sig_gen[:,1],bins=100,density=1.,histtype='step',range=[-10.,10.],label='Flow sig '+condition_str)
    ax_mz1[ix,iy].hist(x_bkg_true[:,1],bins=100,density=1.,histtype='step',range=[-10.,10.],label='True bkg '+condition_str)
    ax_mz1[ix,iy].hist(x_bkg_gen[:,1],bins=100,density=1.,histtype='step',range=[-10.,10.],label='Flow bkg '+condition_str)
    ax_mz1[ix,iy].legend(loc='best')
    ax_mz1[ix,iy].set_title(condition_str)
    
    ax_mz2[ix,iy].hist(x_sig_true[:,2],bins=100,density=1.,histtype='step',range=[-10.,10.],label='True sig'+condition_str)
    ax_mz2[ix,iy].hist(x_sig_gen[:,2],bins=100,density=1.,histtype='step',range=[-10.,10.],label='Flow sig '+condition_str)
    ax_mz2[ix,iy].hist(x_bkg_true[:,2],bins=100,density=1.,histtype='step',range=[-10.,10.],label='True bkg '+condition_str)
    ax_mz2[ix,iy].hist(x_bkg_gen[:,2],bins=100,density=1.,histtype='step',range=[-10.,10.],label='Flow bkg '+condition_str)
    ax_mz2[ix,iy].legend(loc='best')
    ax_mz2[ix,iy].set_title(condition_str)
    
fig_m4l.savefig(os.path.join(output_dir,'m4l.png'))
fig_mz1.savefig(os.path.join(output_dir,'mZ1.png'))
fig_mz2.savefig(os.path.join(output_dir,'mZ2.png'))

In [102]:
# __________________________________________________________________ ||
# Make plots for likelihood
# __________________________________________________________________ ||

import time

n_dim = 1
sig_bkg_ratio = 0.01
bins = 100

bkg_event_size = 5000
sig_event_size = int(sig_bkg_ratio * bkg_event_size)

nf_model.sig.direction = 1
nf_model.sig.direction = 1

fig, ax = plt.subplots(n_dim,n_dim,figsize=figsize)

sig_index = np.random.randint(0,len(sigs),1)[0]
sig = sigs[sig_index]

idx_bkg_batch = np.random.randint(0,bkg.x.shape[0],bkg_event_size)
bkg_x = bkg.x[idx_bkg_batch]

idx_sig_batch = np.random.randint(0,sig.x.shape[0],sig_event_size)
sig_x = sig.x[idx_sig_batch]
sig_cond = sig.condition[idx_sig_batch]

data = np.concatenate([bkg_x,sig_x],axis=0)

plots = [
    ObjDict(name="mll",range=[-10.,10.],bins=100,index=0,histtype='step',),
    ObjDict(name="mZ1",range=[-10.,10.],bins=100,index=1,histtype='step',),
    ObjDict(name="mZ2",range=[-10.,10.],bins=100,index=2,histtype='step',),
]

for p in plots:
    fig, ax = plt.subplots(n_dim,n_dim,figsize=(10,10))
    ax.hist(bkg_x[:,p.index],bins=p.bins,label=p.name+' bkg',range=p.range,histtype=p.histtype)
    ax.hist(sig_x[:,p.index],bins=p.bins,label=p.name+' sig',range=p.range,histtype=p.histtype)
    ax.set_title(str(sig.param))
    fig.savefig(os.path.join(output_dir,str(sig_bkg_ratio)+'_'+p.name+'.png'))
    

In [103]:
p_sig = np.squeeze(nf_model.disc.predict(data))

plot_low = 0.0
plot_high = 0.2
n_grid = 20
x_grid = [plot_low+(plot_high-plot_low)/n_grid*i for i in range(n_grid+1)]

nf_model.sig.direction = -1
condition_concat = np.concatenate([np.ones((data.shape[0],1)) * x for ix,x in enumerate(x_grid)])
x_data_concat = np.concatenate([data for ix,x in enumerate(x_grid)])
z_concat = nf_model.sig.batch_log_loss([x_data_concat,condition_concat])
z_mix = np.zeros(n_grid+1)
z_cond = np.zeros(n_grid+1)

for ig,x in enumerate(x_grid):
    z_cond[ig] = tf.reduce_mean(z_concat[ig*data.shape[0]:(ig+1)*data.shape[0]])
    z_mix[ig] = tf.reduce_mean(np.multiply(z_concat[ig*data.shape[0]:(ig+1)*data.shape[0]],p_sig))

y_scale = 'log'
fig, ax = plt.subplots(2,1,figsize=(10,10))

ax[0].plot(x_grid,z_mix,label="MixtureRealNVP")
ax[0].set_title(str(sig.param))
ax[0].set_yscale(y_scale)
ax[0].legend(loc='best')
ylims = ax[0].get_ylim()
ax[0].arrow(sig.condition[0], ylims[1], 0., ylims[0]-ylims[1],)

ax[1].plot(x_grid,z_cond,label="ConditionalRealNVP")
ax[1].set_title(str(sig.param))
ax[1].legend(loc='best')
ax[1].set_yscale(y_scale)
ylims = ax[1].get_ylim()

ax[1].arrow(sig.condition[0], ylims[1], 0., ylims[0]-ylims[1],)
fig.savefig(os.path.join(output_dir,str(sig_bkg_ratio)+"_log_loss.png"))

for p in plots:
    fig, ax = plt.subplots(n_dim,n_dim,figsize=(10,10))
    ax.hist(data[:,p.index],bins=p.bins,weights=p_sig,label=p.name,range=p.range,histtype=p.histtype)
    fig.savefig(os.path.join(output_dir,str(sig_bkg_ratio)+'_'+p.name+'_weighted.png'))

/Users/lucien/.pyenv/versions/3.7.6/lib/python3.7/site-packages/matplotlib/patches.py:1338: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  verts = np.dot(coords, M) + (x + dx, y + dy)
/Users/lucien/.pyenv/versions/3.7.6/lib/python3.7/site-packages/matplotlib/patches.py:1338: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  verts = np.dot(coords, M) + (x + dx, y + dy)
